<a href="https://colab.research.google.com/github/codedyasai/Python_MachineLearning/blob/main/12_Boosting_%EA%B3%84%EC%97%B4_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

현업에서 주로 사용되는 머신러닝 Algorithm

- LinearRegression
- LogisticRegression
- KNN
- Naive Bayes
- Decision Tree
- Random Forest
- XGBoost
- LightGBM
- KMeans
- PCA

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install mglearn
import mglearn

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 7.0 MB/s eta 0:00:00


**정형 데이터**: 행과 열로 나눠 데이터베이스나 엑셀 파일(csv)파일로 저장할 수 있는 데이터
- 머신러닝 학습용
- 특성공학을 적용하는 전처리 단계에서 필요한 데이터

**비정형 데이터**: 정형 데이터에 속하지 않은 데이터로, 텍스트, 오디오, 이미지, 영상 등
- 딥러닝 학습용
- 보통 특성공학 대신에 표현학습 알고리즘을 적용

# 랜덤 포레스트

- 정형 데이터를 다루는 대표적인 머신러닝 알고리즘
- 결정 트리를 무작위로 만들어 숲을 만드는 알고리즘
- CART 계열: RandomForestClassifier, RandomForestRegressor
- 부트스트랩 샘플로 훈련을 진행한다.
- 부트스트랩 샘플이란 훈련 세트에서 중복을 허용해 무작위로 샘플링한 데이터 샘플을 뜻한다.
- 각각의 부트스트랩 샘플들로 훈련한 모델들의 예측 정확도를 모두 더해 트리의 개수로 나누면 평균 정확도가 산출된다.
- 특성 개수의 루트만큼 특성을 무작위로 골라 최선의 특성 분할 방법을 찾아서 훈련한다.
- 모든 노드를 만들 때마다 이 과정을 반복해 최선의 특성 분할을 적용하는 알고리즘

In [ ]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy() # 사이킷런으로 작업하려면 array로 바꿔줘야 한다
target = wine[['class']].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target, = train_test_split(
    data, target, test_size= 0.2 , random_state=42)

In [ ]:
train_input.shape, test_input.shape

((5197, 3), (1300, 3))

In [ ]:
train_target.shape, test_target.shape

((5197, 1), (1300, 1))

**분류 알고리즘을 사용해 분류할 때 그 찾아낸 대상을 양성클래스라고 부르고, 1의 정형값을 갖는다**

# 랜덤 포레스트 분류기를 이용한 화이트와인 분류

In [ ]:
from scipy.sparse import random
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# n_jobs= -1 : 시스템의 CPU 코어를 모두 사용한다.
rf = RandomForestClassifier(n_jobs= -1, random_state= 42)

# return_train_score= True: 검증세트 뿐만 아니라 훈련세트의 점수도 같이 반환한다
scores = cross_validate(rf, train_input, train_target, n_jobs= -1, return_train_score= True)


print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
# oob: 부트스트랩 샘플을 통해 샘플링한 데이터 중에서도 전혀 훈련에 사용되지 않은 데이터.
# 모델 자체 평가: oob_score= True
rf = RandomForestClassifier(oob_score= True, n_jobs= -1, random_state= 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리(Extra Tree)

- 랜덤 포레스트와 비슷한 모델
- 랜덤 포레스트와 거의 모든 과정이 비슷하지만 과대적합을 억제하는 방식에 차이가 있다
- 부트스트랩 샘플개신 전체 샘플을 사용한다.
- 여러 특성을 기준으로 랜덤하게 트리를 분할하고, 이중 최적의 불순도를 가지는 트리를 찾아내는 방법
- 그래서 랜덤 포레스트에 비해 속도가 빠르다.
- ExtraTreesClassfier 클래스를 사용
(사실 현업에선 잘 안쓴다고 한다)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs= -1, random_state= 42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score= True, n_jobs= -1)

In [ ]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 위에 한것에 비해 테스트 점수가 좀 떨어짐

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


- 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 한다.
- 특성의 갯수도 더 많이 주어질 때 랜덤 포레스트보다는 유리한 성능을 제공한다.

GradientBoostingClassifier,
# 그래디언트 부스팅(Gradient Boosting)

- 기본적으로 깊이가 3인 결정 트리 100개를 사용
- 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대
- 분류에서는 로지스틱 손실함수, 회귀에서는 평균제곱오차를 사용한다.
- 이전 트리의 손실 값들이 낮아지는 방향으로 회귀 결정 트리를 계속 추가하는 알고리즘
- GradientBoostingClassifier/Regressor 클래스
- max_depth 또는 learning_rate 파라미터를 추가해 과대적합을 억제할 수 있다.
- 학습률을 증가시키고 트리의 개수를 늘리면 좀 더 성능이 향상될 수 있다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state= 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 트리의 개수를 늘려야 할 것 같다.

0.8881086892152563 0.8720430147331015


In [ ]:
# n_estimators: 트리의 개수, 기본값 100
# learning_rate: 학습율, 기본값 0.1

gb = GradientBoostingClassifier( n_estimators= 500, learning_rate= 0.2, random_state= 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# 히스토그램 기반 그래디언트 부스팅(HGBM)

- Histogram-base Gradient Boosting: 정형 데이터를 다루는 머신러닝 알고리즘에서 가장 인기가 높은 알고리즘
- 훈련 데이터를 256개의 구간으로 나눠 학습시키는 알고리즘
- 트리의 개수를 지정하는 n_estimators 파라미터 대신 부스팅 횟수 지정하는 max_iter를 사용
- 성능을 향상시키기 위해 max_iter 파라미터를 조정한다.

## 버전 확인

In [ ]:
import sys
# 파이썬 버전을 확인할 때
print('Python: ', sys.version)
print('Pandas: ', pd.__version__)
print('Numpy: ', np.__version__)

import sklearn
print('Scikit-learn: ', sklearn.__version__)


Python:  3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
Pandas:  1.5.3
Numpy:  1.23.5
Scikit-learn:  1.2.2


In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state= 42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
# feature의 중요도를 별도의 함수를 이용하여 계산을 수행한다.
from sklearn.inspection import permutation_importance

# permutatuon_importance, 반환하는 객체는 반봇해서 얻은 특성 중요도(importances),
# 평균(importances_mean), 표준편차(importances_std)를 담고 있다.
hgb.fit(train_input, train_target)

# n_repeats= 10, 랜덤하게 섞을 횟수
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats= 10, random_state= 42, n_jobs= -1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats= 10, random_state= 42, n_jobs= -1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

# XGBoost vs LightGBM

- 히스토그램 그래디언트 부스팅 전용 라이브러리가 XGBoost, LightGBM, NGBoost...
- 트리 기반의 알고리즘의 앙상블 학습에서 각광받는 알고리즘중 하나
- GBM에 기반하고 있지만, GBM의 단점인 느린 수행시간, 과적합 규제등을 해결한 알고리즘

**주요 장점**
- 뛰어난 예측성능
- GBM 대비 빠른 수행시간
- 과적합 규제
- Tree pruning(트리 가지치기): 긍정 이득이 없는 분할을 가지치기해서 분할 수 줄임
- 자체 내장된 교차검증
  - 반복 수행시마다 내부적으로 교차검증을 수행해 최적화된 반복 수행횟수를 가질 수 있다.
  - 지정된 반복횟수가 아니라 교차검증을 통해 평가 데이터세트의 평가 값이 최적화되면 반복을 중간에 멈출 수 있는 기능이 있다.
- 결손값 자체 처리
- XGBoostsms 독자적인 XGBoost 모듈과 사이킷런 프레임워크 기반의 모듈이 존재한다.
- 독자적인 모듈은 고유의 API와 하이퍼파라미터를 사용하지만, 사이킷런 기반 모듈에서는 다른 Estimator와 동일한 사용법을 가진다.

In [ ]:
!pip install xgboost

import xgboost
print(xgboost.__version__)

2.0.1


In [ ]:
from xgboost import XGBClassifier

xgb =XGBClassifier(tree_method= 'hist', random_state= 42)

scores = cross_validate(xgb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [ ]:
! pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

lgb =LGBMClassifier(tree_method= 'hist', random_state= 42)

scores = cross_validate(lgb, train_input, train_target,
                        return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))